In [27]:
import pandas as pd
from datetime import datetime
import boto3
import json
import io

pd.set_option('display.max_columns', None)


S3_BUCKET_SOURCE = 'imdb-etl-silver-whoortuydmo9'

In [15]:
s3 = boto3.client("s3")

def list_csv_object(s3, bucket, prefix):
        paginator = s3.get_paginator("list_objects_v2")
        result = []
        for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
            for obj in page.get("Contents", []):
                if obj["Key"].endswith(".csv"):
                    result.append(obj["Key"])
        return result


object_keys = list_csv_object(s3, S3_BUCKET_SOURCE, "silver/movies_normalized")
object_keys

['silver/movies_normalized.csv']

In [ ]:
def load_csv(s3, bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    return pd.read_csv(io.BytesIO(response['Body'].read()))

normalized_data = load_csv(s3, S3_BUCKET_SOURCE,'silver/movies_normalized.csv')
normalized_data

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [29]:
import io

# Read a CSV file from the S3 bucket using pandas

key = object_keys[0]
response = s3.get_object(Bucket=S3_BUCKET_SOURCE, Key=key)
df_from_s3 = pd.read_csv(io.BytesIO(response['Body'].read()))
df_from_s3.head()

,id,rank,title,fulltitle,year,image,crew,imdbrating,imdbratingcount,rated,released,runtime,genre,director,writer,actors,plot,language,country,awards,poster,ratings,metascore,imdbvotes,imdbid,type,dvd,boxoffice,production,website,response
0,tt0050083,5,12 Angry Men,12 Angry Men (1957),1957,https://m.media-amazon.com/images/M/MV5BMWU4N2...,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",9.0,759763,Approved,10 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,Reginald Rose,"Henry Fonda, Lee J. Cobb, Martin Balsam",The jury in a New York City murder trial is fr...,English,United States,Nominated for 3 Oscars. 16 wins & 12 nominatio...,https://m.media-amazon.com/images/M/MV5BYjE4Nz...,"[{'Source': 'Internet Movie Database', 'Value'...",97,"931,674",tt0050083,movie,NaN,NaN,NaN,NaN,True
1,tt0060196,10,"The Good, the Bad and the Ugly","The Good, the Bad and the Ugly (1966)",1966,https://m.media-amazon.com/images/M/MV5BNjJlYm...,"Sergio Leone (dir.), Clint Eastwood, Eli Wallach",8.8,739563,R,29 Dec 1967,178 min,"Adventure, Drama, Western",Sergio Leone,"Luciano Vincenzoni, Sergio Leone, Agenore Incr...","Clint Eastwood, Eli Wallach, Lee Van Cleef",A bounty-hunting scam joins two men in an unea...,"Italian, English","Italy, Spain, West Germany, United States",2 wins & 5 nominations total,https://m.media-amazon.com/images/M/MV5BMWM5Zj...,"[{'Source': 'Internet Movie Database', 'Value'...",90,"857,148",tt0060196,movie,NaN,"$25,100,000",NaN,NaN,True
2,tt0068646,2,The Godfather,The Godfather (1972),1972,https://m.media-amazon.com/images/M/MV5BM2MyNj...,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.2,1771457,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",United States,Won 3 Oscars. 31 wins & 31 nominations total,https://m.media-amazon.com/images/M/MV5BNGEwYj...,"[{'Source': 'Internet Movie Database', 'Value'...",100,"2,132,118",tt0068646,movie,NaN,"$136,381,073",NaN,NaN,True
3,tt0071562,4,The Godfather Part II,The Godfather: Part II (1974),1974,https://m.media-amazon.com/images/M/MV5BMWMwMG...,"Francis Ford Coppola (dir.), Al Pacino, Robert...",9.0,1225028,R,18 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo","Al Pacino, Robert De Niro, Robert Duvall",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",United States,Won 6 Oscars. 17 wins & 21 nominations total,https://m.media-amazon.com/images/M/MV5BMDIxMz...,"[{'Source': 'Internet Movie Database', 'Value'...",90,"1,429,496",tt0071562,movie,NaN,"$47,834,595",NaN,NaN,True
4,tt0108052,6,Schindler's List,Schindler's List (1993),1993,https://m.media-amazon.com/images/M/MV5BNDE4OT...,"Steven Spielberg (dir.), Liam Neeson, Ralph Fi...",9.0,1310450,R,04 Feb 1994,195 min,"Biography, Drama, History",Steven Spielberg,"Thomas Keneally, Steven Zaillian","Liam Neeson, Ralph Fiennes, Ben Kingsley","In German-occupied Poland during World War II,...","English, Hebrew, German, Polish, Latin",United States,Won 7 Oscars. 91 wins & 49 nominations total,https://m.media-amazon.com/images/M/MV5BNjM1ZD...,"[{'Source': 'Internet Movie Database', 'Value'...",95,"1,522,278",tt0108052,movie,NaN,"$96,898,818",NaN,NaN,True


In [ ]:

def process(self, prefix):
    object_keys = self.s3.list_csv_object(self.source_bucket, prefix)
    if not object_keys:
        raise Exception(f"No .csv files found under {prefix}")

    normalized_data = [self.s3.load_csvs(self.source_bucket, key) for key in object_keys]
    df = self.transform_to_df(self, normalized_data)
    self.process_analytics(self, df)

    return len(df)

def transform_to_df(self, normalized_data):
    df = pd.DataFrame(normalized_data)
    if df.empty:
        raise Exception("No data to process after transformation!")
    
    df['rank'] = df['rank'].astype(int)
    df = df.sort_values(by='rank')

    return df

def process_analytics(self, df):
    prefix = "gold/"

    try:
        # 1. Top N IMDb Ratings
        topN_rated_df = (
            df[[
                'rank', 'title', 'year', 'imdbrating', 'imdbratingcount',
                'released', 'runtime', 'genre', 'director', 'language',
                'country', 'awards', 'metascore', 'imdbvotes', 'boxoffice'
            ]]
            .dropna()
            .sort_values(by='imdbrating', ascending=False)
            .head(10)
        )

        # 2. Genre Distribution
        genre_series = df['genre'].dropna().str.split(', ').explode()
        movies_by_genre_df = genre_series.value_counts().rename_axis('genre').reset_index(name='count')

        # 3. movies per country
        df['country'] = df['country'].fillna('').astype(str)
        countries_exploded = df['country'].str.split(',', expand=True).stack().str.strip()
        country_counts = countries_exploded.value_counts().reset_index()
        country_counts.columns = ['Country', 'MovieCount']
        country_counts = country_counts.sort_values(by='MovieCount', ascending=False)

        # 4. Movies per year
        movies_per_year_df = (
            df['year'].dropna().astype(int).value_counts().sort_index(ascending=False).rename_axis('year').reset_index(name='count')
        )
        
        # 5. Total Box Office Revenue per Year
        df['boxoffice_clean'] = (
            df['boxoffice'].astype(str)
            .str.replace('$', '', regex=False)
            .str.replace(',', '', regex=False)  
            .replace('nan', '0') 
            .astype(int)
        )
        box_office_df = (
            df.groupby('year', as_index=False)['boxoffice_clean']
            .sum()
            .rename(columns={'boxoffice_clean': 'total_box_office'})
            .sort_values(by='year', ascending=False)
        )

        # 6. Top 5 Directors by Movie Count
        director_series = df['director'].dropna().str.split(', ').explode()
        top_directors_df = (
            director_series.value_counts().head(5).rename_axis('director').reset_index(name='movie_count')
        )

    except Exception as e:
        raise Exception(f"Error processing analytics: {e}")

In [4]:
processor = SilverToGoldProcessor()

data = pd.read_csv('movies_normalized.csv')


In [ ]:
print('Total processed:' processor(data))